# Simpsons characters dialogs

In [151]:
import pandas as pd
import numpy as np
from flask import Flask, request
from pymongo import MongoClient
from bson.objectid import ObjectId
from bson.json_util import dumps
import ast
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity as distance
import seaborn as sns

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/TryAgain/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [152]:
sia = SentimentIntensityAnalyzer()

In [153]:
client = MongoClient("mongodb://localhost/Conversations")
db = client.get_database()

In [74]:
df = pd.read_csv('input/simpsons_dataset.csv', engine = 'python')
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [75]:
nombres = { 
    'raw_character_text':'Characters'
}
df = df.rename(columns=nombres)

# Create characters groups

In [76]:
df = df = df.dropna()
df = df[(df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson', 'Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince', 'Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson')))]
df.head()

,Characters,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
5,Martin Prince,I don't think there's anything left to say.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!


In [77]:
df['Groups'] = df['Groups'][df['Characters'].isin(('Homer Simpson', 'Marge Simpson', 'Bart Simpson', 'Lisa Simpson', 'Maggie Simpson', 'Abraham Simpson'))] = 'Home'
df['Groups'][df['Characters'].isin(('Milhouse Van Houten', 'Ralph Wiggum', 'Nelson Muntz', 'Seymour Skinner', 'Edna Krabappel', 'Groundskeeper Willie', 'Jimbo Jones', 'Martin Prince'))] = 'School'
df['Groups'][df['Characters'].isin(('Moe Szyslak', 'Barney Gumble', 'Lenny Leonard', 'Carl Carlson'))] = 'Bar'

In [78]:
df_reduced = df.head(100)
df_reduced.head()

,Characters,spoken_words,Groups
1,Lisa Simpson,Where's Mr. Bergstrom?,Home
3,Lisa Simpson,That life is worth living.,Home
5,Martin Prince,I don't think there's anything left to say.,School
7,Bart Simpson,Victory party under the slide!,Home
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!,Home


In [79]:
ch = df[['Characters', 'Groups']].drop_duplicates()

# Including Simpsons characters on MongoDB

In [80]:
import requests

In [14]:
for index, row in ch.iterrows():
    data = requests.get(f"http://localhost:8800/user/create/{row['Characters']}/{row['Groups']}").content
print(data)

b'{\n  "user_id": "5e52c726b6fcdc030722c3cd"\n}\n'


# Creating chats

In [15]:
for group in pd.unique(df_reduced['Groups']):
    data = requests.get(f"http://localhost:8800/chat/create/{group}").content
print(data)

b'{\n  "Group": "Bar"\n}\n'


# Adding characters to chat

In [18]:

for character in pd.unique(df_reduced['Characters']):
    print(character)
    usersdata = requests.get(f"http://localhost:8800/chat/adduser/{character}").text
    print(usersdata)

Lisa Simpson
User added to the chat
Martin Prince
User added to the chat
Bart Simpson
User added to the chat
Nelson Muntz
User added to the chat
Milhouse Van Houten
User added to the chat
Homer Simpson
User added to the chat
Marge Simpson
User added to the chat
Moe Szyslak
User added to the chat


# Adding messages to chat

In [19]:
'''
Characters from the same group will be on the same MongoDB document conversation
'''
for index, row in df_reduced.iterrows():  
    data = requests.get(f"http://localhost:8800/chat/{row['Groups']}/{row['Characters']}/{row['spoken_words']}").content
print(data)

b'Great. Included'


# Getting messages from chat

In [23]:
#If you wanna have username + message:
groups = db['Conversations'].distinct('Group')

for group in groups:
    conversations = requests.get(f"http://localhost:8800/chat/{group}/list").text

    print(conversations)



[{"username": "Moe Szyslak", "message": "College boy."}]
[{"username": "Lisa Simpson", "message": "Where's Mr. Bergstrom"}, {"username": "Lisa Simpson", "message": "That life is worth living."}, {"username": "Bart Simpson", "message": "Victory party under the slide!"}, {"username": "Lisa Simpson", "message": "Mr. Bergstrom! Mr. Bergstrom!"}, {"username": "Lisa Simpson", "message": "Do you know where I could find him"}, {"username": "Lisa Simpson", "message": "The train, how like him... traditional, yet environmentally sound."}, {"username": "Lisa Simpson", "message": "I see he touched you, too."}, {"username": "Bart Simpson", "message": "Hey, thanks for your vote, man."}, {"username": "Bart Simpson", "message": "Well, you got that right. Thanks for your vote, girls."}, {"username": "Bart Simpson", "message": "Well, don't sweat it. Just so long as a couple of people did... right, Milhouse"}, {"username": "Bart Simpson", "message": "Lewis"}, {"username": "Bart Simpson", "message": "Someb

In [24]:
#If you wanna have a list with messages:
groups = db['Conversations'].distinct('Group')

for group in groups:
    messages = requests.get(f"http://localhost:8800/chat/{group}/userlist").text

    print(messages)

["College boy."]
["Where's Mr. Bergstrom", "That life is worth living.", "Victory party under the slide!", "Mr. Bergstrom! Mr. Bergstrom!", "Do you know where I could find him", "The train, how like him... traditional, yet environmentally sound.", "I see he touched you, too.", "Hey, thanks for your vote, man.", "Well, you got that right. Thanks for your vote, girls.", "Well, don't sweat it. Just so long as a couple of people did... right, Milhouse", "Lewis", "Somebody must have voted.", "Uh oh.", "I demand a recount.", "No.", "Mr. Bergstrom! Hey, Mr. Bergstrom!", "Hey, Lisa, indeed.", "Oh, I mean, were you just going to leave, just like that", "You can't go! You're the best teacher I'll ever have.", "Oh, please.", "But I need you too.", "I, I understand. Mr. Bergstrom, I'm going to miss you.", "Thank you, Mr. Bergstrom.", "So, I guess this is it", "Never thrown a party", "Bart didn't get one vote", "Nothing.", "Lisa, tell your father.", "Mr. Bergstrom left today.", "Oh.", "He's gone. F

# Sentiment analysis

In [28]:
#Group sentiment analysis
groups = db['Conversations'].distinct('Group')
for group in groups:
    sentiment = requests.get(f"http://localhost:8800/chat/{group}/sentimentgroup").text

    print('For the group ',group,' the sentiment is ', sentiment)

For the group  Bar  the sentiment is  {
  "compound": 0.0, 
  "neg": 0.0, 
  "neu": 1.0, 
  "pos": 0.0
}

For the group  Home  the sentiment is  {
  "compound": 0.9927, 
  "neg": 0.052, 
  "neu": 0.836, 
  "pos": 0.112
}

For the group  School  the sentiment is  {
  "compound": 0.0, 
  "neg": 0.0, 
  "neu": 1.0, 
  "pos": 0.0
}



In [169]:
#Error. Revisar
#Character sentiment analysis

characters = [character['username'] for character in list(db['Conversations'].distinct('Characters'))]
for character in characters:
    sentimentCharac = requests.get(f"http://localhost:8800/chat/{character}/sentimentcharacter").text
    print(character, sentimentCharac)

Bart Simpson {
  "compound": 0.8308, 
  "neg": 0.052, 
  "neu": 0.801, 
  "pos": 0.148
}

Homer Simpson {
  "compound": 0.9643, 
  "neg": 0.061, 
  "neu": 0.818, 
  "pos": 0.12
}

Lisa Simpson {
  "compound": 0.954, 
  "neg": 0.048, 
  "neu": 0.792, 
  "pos": 0.16
}

Marge Simpson {
  "compound": 0.8321, 
  "neg": 0.095, 
  "neu": 0.757, 
  "pos": 0.148
}

Martin Prince {
  "compound": 0.0, 
  "neg": 0.0, 
  "neu": 0.0, 
  "pos": 0.0
}

Milhouse Van Houten {
  "compound": 0.0, 
  "neg": 0.0, 
  "neu": 0.0, 
  "pos": 0.0
}

Moe Szyslak {
  "compound": 0.0, 
  "neg": 0.0, 
  "neu": 0.0, 
  "pos": 0.0
}

Nelson Muntz {
  "compound": 0.0, 
  "neg": 0.0, 
  "neu": 0.0, 
  "pos": 0.0
}



In [168]:
character='Lisa Simpson'

sentimentCharac = requests.get(f"http://localhost:8800/chat/{character}/sentimentcharacter").text
print(character, sentimentCharac)

Lisa Simpson {
  "compound": 0.954, 
  "neg": 0.048, 
  "neu": 0.792, 
  "pos": 0.16
}



# Recommending an user

In [171]:
characters = [character['username'] for character in list(db['Conversations'].distinct('Characters'))]
for username in characters:
    recomCharac = requests.get(f"http://localhost:8800/user/{username}/recommend").text
    print('For ', username, ' we recommend ', recomCharac)

For  Bart Simpson  we recommend  Homer Simpson
For  Homer Simpson  we recommend  Lisa Simpson
For  Lisa Simpson  we recommend  Homer Simpson
For  Marge Simpson  we recommend  Homer Simpson
For  Martin Prince  we recommend  Homer Simpson
For  Milhouse Van Houten  we recommend  Homer Simpson
For  Moe Szyslak  we recommend  Bart Simpson
For  Nelson Muntz  we recommend  Bart Simpson
